In [2]:
from langchain_core.output_parsers  import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "AI"})

"Why did the artificial intelligence break up with its robot girlfriend? It couldn't handle her constant buffering."

In [5]:
print(prompt.invoke({"topic":"AI"}))

messages=[HumanMessage(content='tell me a short joke about AI')]


In [7]:
from langchain_core.messages.human import HumanMessage
messages =[HumanMessage(content='tell me a short joke about AI')]
model.invoke(messages)

AIMessage(content='Why did the AI break up with its computer program girlfriend? \nBecause she had too many bugs in her code!', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 14, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-534361ce-a166-469d-8749-574591501f1b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 23, 'total_tokens': 37})